In [1]:
import os
import requests
from dotenv import load_dotenv

In [2]:
load_dotenv()

API_KEY = os.getenv('UNISWAP_API_KEY') # Fetching the Etherscan API key from environment variables

if not API_KEY: 
    raise ValueError('Please set the UNISWAP_API_KEY environment variable.')

In [ ]:
# EOA → Contract: typical user calling a protocol

# Contract → Contract: smart contract calling Uniswap V3 Router

address = "0xE592427A0AEce92De3Edee1F18E0157C05861564" # Uniswap V3 Router contract address

# Helper function to get transactions
def get_transactions(action="txlist", chainid=1, offset=1000, page=1):
    url = (
        f"https://api.etherscan.io/v2/api"
        f"?chainid={chainid}"
        f"&module=account"
        f"&action={action}"
        f"&address={address}"
        f"&startblock=0"
        f"&endblock=latest"
        f"&page={page}"
        f"&offset={offset}"
        f"&sort=desc"
        f"&apikey={API_KEY}"
    )
    response = requests.get(url).json()
    return response["result"]

def is_contract(address):
    url = f"https://api.etherscan.io/api?module=proxy&action=eth_getCode&address={address}&tag=latest&apikey={API_KEY}"
    response = requests.get(url).json()
    return response["result"] != "0x"

# Normal transactions
def transactions(txs):
    normal = []
    for tx in txs:
        from_addr = tx["from"]
        to_addr = tx["to"]
        tx["from_type"] = "Contract" if is_contract(from_addr) else "EOA"
        tx["to_type"]   = "Contract" if is_contract(to_addr) else "EOA"
        normal.append(tx)
    return normal

# Example usage
normal_uniswap_tx = get_transactions("txlist", offset=5)  # last 5 normal transactions
uniswap_n_tx = transactions(normal_uniswap_tx)

for tx in uniswap_n_tx:
    print(
        f"TxHash: {tx['hash']} | From: {tx['from']} | {tx['from_type']} | To: {tx['to']} | {tx['to_type']}"
    )

TxHash: 0x9e46e0a36a4c48003fdf8e190f5b454ad999b22e7682bbf02b47d7ee10be26db | From: 0xca2d8dba4710cbb1b87ae7aeffac53013be03387 | EOA | To: 0xe592427a0aece92de3edee1f18e0157c05861564 | Contract
TxHash: 0x3d4742bfaf9ad0b5f90e1521c111b4f6bf69ecccac26b6c4af0f1a9b8ba9dae9 | From: 0x604dcf973f647c1d6d5169d68240cf870b03701a | Contract | To: 0xe592427a0aece92de3edee1f18e0157c05861564 | Contract
TxHash: 0x2e74d849d611c58ad9d05bfafebafc1fde8a707800cd5d8443b15f2a0cfa064f | From: 0xcc98820ce1ae898e81e1fd483f44616fec0779a9 | EOA | To: 0xe592427a0aece92de3edee1f18e0157c05861564 | Contract
TxHash: 0x5af029d9db69482f52ead068d642b783b020d55deb8007e9bfd7b80cdd1ada54 | From: 0x39cfae8723739e93bc186d06f9ee238089e59b39 | EOA | To: 0xe592427a0aece92de3edee1f18e0157c05861564 | Contract
TxHash: 0x6abe11840ca97d092d94328020a1b9d9ed1087f104c9b362ef8507528a2f90a6 | From: 0xf5213a6a2f0890321712520b8048d9886c1a9900 | EOA | To: 0xe592427a0aece92de3edee1f18e0157c05861564 | Contract


In [4]:
user_to_contract = [tx for tx in uniswap_n_tx if tx["from_type"] == "EOA" and tx["to_type"] == "Contract"]
user_to_contract

[{'blockNumber': '23346241',
  'timeStamp': '1757671163',
  'hash': '0x9e46e0a36a4c48003fdf8e190f5b454ad999b22e7682bbf02b47d7ee10be26db',
  'nonce': '1030',
  'blockHash': '0x11d26facf51eab3366cc05e1a3567c01b77c1441c0bb8d841f59fc3e76e0a328',
  'transactionIndex': '7',
  'from': '0xca2d8dba4710cbb1b87ae7aeffac53013be03387',
  'to': '0xe592427a0aece92de3edee1f18e0157c05861564',
  'value': '0',
  'gas': '250000',
  'gasPrice': '1454237634',
  'isError': '1',
  'txreceipt_status': '0',
  'input': '0x414bf38900000000000000000000000070e8de73ce538da2beed35d14187f6959a8eca96000000000000000000000000a0b86991c6218b36c1d19d4a2e9eb0ce3606eb4800000000000000000000000000000000000000000000000000000000000001f4000000000000000000000000ca2d8dba4710cbb1b87ae7aeffac53013be033870000000000000000000000000000000000000000000000000000000068c3f011000000000000000000000000000000000000000000000000000000004d7c6d00000000000000000000000000000000000000000000000000000000003e2434000000000000000000000000000000000000000000000

In [5]:
contract_to_contract = [tx for tx in uniswap_n_tx if tx["from_type"] == "Contract" and tx["to_type"] == "Contract"]
contract_to_contract

[{'blockNumber': '23346237',
  'timeStamp': '1757671115',
  'hash': '0x3d4742bfaf9ad0b5f90e1521c111b4f6bf69ecccac26b6c4af0f1a9b8ba9dae9',
  'nonce': '81',
  'blockHash': '0x38099c826614984029844f056a0a2f636094282775cad989625a3cfd1fdb05c6',
  'transactionIndex': '2',
  'from': '0x604dcf973f647c1d6d5169d68240cf870b03701a',
  'to': '0xe592427a0aece92de3edee1f18e0157c05861564',
  'value': '11000000000000000',
  'gas': '171685',
  'gasPrice': '2282464799',
  'isError': '0',
  'txreceipt_status': '1',
  'input': '0x414bf389000000000000000000000000c02aaa39b223fe8d0a0e5c4f27ead9083c756cc2000000000000000000000000b56aaac80c931161548a49181c9e000a19489c440000000000000000000000000000000000000000000000000000000000000bb8000000000000000000000000604dcf973f647c1d6d5169d68240cf870b03701a0000000000000000000000000000000000000000000000000000000068c3f11700000000000000000000000000000000000000000000000000271471148780000000000000000000000000000000000000000000000000000f3071f37aaa400000000000000000000000000000000

In [ ]:
# An internal transaction isn’t a real transaction broadcast to the chain, It’s a log of value transfers or contract calls that happen inside the execution of a top-level transaction.
# Example: Alice (EOA) calls Uniswap Router --> The Router calls a Pool contract --> The Pool sends ETH back to Alice.
# Contract --> EOA: That happens when a contract sends ETH or tokens back to a wallet inside the execution.

address = "0xE592427A0AEce92De3Edee1F18E0157C05861564"  # Uniswap V3 Router

def get_transactions(action="txlistinternal", chainid=1, offset=1000, page=1):
    url = (
        f"https://api.etherscan.io/v2/api"
        f"?chainid={chainid}"
        f"&module=account"
        f"&action={action}"
        f"&address={address}"
        f"&startblock=0"
        f"&endblock=latest"
        f"&page={page}"
        f"&offset={offset}"
        f"&sort=desc"
        f"&apikey={API_KEY}"
    )
    response = requests.get(url).json()
    return response["result"]

# Contract check
def is_contract(address):
    url = f"https://api.etherscan.io/api?module=proxy&action=eth_getCode&address={address}&tag=latest&apikey={API_KEY}"
    response = requests.get(url).json()
    return response["result"] != "0x"

# Enrichment
def internal_transactions(txs):
    internal = []
    for tx in txs:
        from_addr = tx.get("from")
        to_addr   = tx.get("to")
        tx["from_type"] = "Contract" if is_contract(from_addr) else "EOA"
        tx["to_type"]   = "Contract" if to_addr and is_contract(to_addr) else "EOA"
        internal.append(tx)
    return internal

# Example: get last 5 internal transactions
internal_uniswap_tx = get_transactions("txlistinternal", offset=5)
uniswap_internal = internal_transactions(internal_uniswap_tx)

for tx in uniswap_internal:
    print(
        f"TxHash: {tx['hash']} | From: {tx['from']} | {tx['from_type']} | To: {tx['to']} | {tx['to_type']}"
    )


TxHash: 0x455e80723ef63585c3beb7a62bb34ae2a1b6f7a41c407c18135ed9c1a8a69fc8 | From: 0xe592427a0aece92de3edee1f18e0157c05861564 | Contract | To: 0x7ee2ee8e1ba4a109fcf8fb741a7fc0d037a1b7ea | EOA
TxHash: 0x455e80723ef63585c3beb7a62bb34ae2a1b6f7a41c407c18135ed9c1a8a69fc8 | From: 0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2 | Contract | To: 0xe592427a0aece92de3edee1f18e0157c05861564 | Contract
TxHash: 0x91a4aedab26ad0d88abf740576516d1fddf007db381f8325a125aac33cba45b4 | From: 0xe592427a0aece92de3edee1f18e0157c05861564 | Contract | To: 0x9eb13eed7348f3c66acf7628f27d89e727e276d8 | EOA
TxHash: 0x91a4aedab26ad0d88abf740576516d1fddf007db381f8325a125aac33cba45b4 | From: 0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2 | Contract | To: 0xe592427a0aece92de3edee1f18e0157c05861564 | Contract
TxHash: 0x2a9b01d542d4c19965f7d4f71a2d0eee2a15c237fa91e0ef465e195da5a5707d | From: 0xe592427a0aece92de3edee1f18e0157c05861564 | Contract | To: 0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2 | Contract


In [9]:
address = "0xE592427A0AEce92De3Edee1F18E0157C05861564"  # Uniswap V3 Router

def get_token_transactions(action="tokentx", chainid=1, offset=1000, page=1):
    url = (
        f"https://api.etherscan.io/v2/api"
        f"?chainid={chainid}"
        f"&module=account"
        f"&action={action}"
        f"&address={address}"
        f"&startblock=0"
        f"&endblock=latest"
        f"&page={page}"
        f"&offset={offset}"
        f"&sort=desc"
        f"&apikey={API_KEY}"
    )
    response = requests.get(url).json()
    return response["result"]

def is_contract(addr):
    url = f"https://api.etherscan.io/api?module=proxy&action=eth_getCode&address={addr}&tag=latest&apikey={API_KEY}"
    resp = requests.get(url).json()
    return resp["result"] != "0x"

def token_txs(txs):
    token = []
    for tx in txs:
        from_addr = tx["from"]
        to_addr = tx["to"]
        tx["from_type"] = "Contract" if is_contract(from_addr) else "EOA"
        tx["to_type"]   = "Contract" if is_contract(to_addr) else "EOA"

        # scale token value using tokenDecimal
        decimals = int(tx.get("tokenDecimal", 0))
        if decimals > 0:
            tx["value_scaled"] = int(tx["value"]) / (10 ** decimals)
        else:
            tx["value_scaled"] = tx["value"]

        token.append(tx)
    return token

uniswap_token_tx = get_token_transactions("tokentx", offset=5)        # last 5 token transfers
uniswap_tokens = token_txs(uniswap_token_tx)

for tx in uniswap_tokens:
    print(
        f"TxHash: {tx['hash']} | Token: {tx['tokenSymbol']} | "
        f"From: {tx['from']} ({tx['from_type']}) | "
        f"To: {tx['to']} ({tx['to_type']}) | "
        f"Value: {tx['value_scaled']}"
    )

TxHash: 0xfd09ff1727f23b660c0736e4fce339a62e84f02b058dfccb05ec14c3091b622c | Token: WETH | From: 0xe592427a0aece92de3edee1f18e0157c05861564 (Contract) | To: 0xd19393e02c3cc60c21a89b5e85656ff1122ebd51 (Contract) | Value: 0.4327860059182891
TxHash: 0x1ef5985ac991d3940b3205cf8992d2b51a70e516e389336b8c7ede20bb47ce65 | Token: WETH | From: 0xc7bbec68d12a0d1830360f8ec58fa599ba1b0e9b (Contract) | To: 0xe592427a0aece92de3edee1f18e0157c05861564 (Contract) | Value: 5.744855793164275
TxHash: 0x282449f7cd1961a725f8bc875c63212c66690f6371c29c669e4b8d78218049d1 | Token: WETH | From: 0x82c427adfdf2d245ec51d8046b41c4ee87f0d29c (Contract) | To: 0xe592427a0aece92de3edee1f18e0157c05861564 (Contract) | Value: 0.06377028086316641
TxHash: 0xfc14290564aec2bd3f54e152dd94857569ff76ff29e06cd238f7fba52a755e5c | Token: WETH | From: 0xe592427a0aece92de3edee1f18e0157c05861564 (Contract) | To: 0xd6dc7e6887dd119a9dafa69cd0004e2c55b2aafc (Contract) | Value: 0.02952
TxHash: 0x03951077ffd15213288dc76aff2c9813d9f27b4981101

In [ ]:
# Fetch data
normal_uniswap_tx = get_transactions("txlist")
internal_uniswap_tx = get_transactions("txlistinternal")
uniswap_token_tx = get_transactions("tokentx")

# Combine and classify
combined = []

for tx in normal_tx:
    combined.append({
        "txHash": tx['hash'],
        "from": tx['from'],
        "to": tx['to'],
        "value": tx['value'],
        "type_from": "Contract" if is_contract(tx['from']) else "EOA",
        "type_to": "Contract" if is_contract(tx['to']) else "EOA",
        "internal": False,
        "tokenTransfer": False,
        "blockNumber": tx['blockNumber']
    })

for tx in internal_tx:
    combined.append({
        "txHash": tx['hash'],
        "from": tx['from'],
        "to": tx['to'],
        "value": tx['value'],
        "type_from": "Contract" if is_contract(tx['from']) else "EOA",
        "type_to": "Contract" if is_contract(tx['to']) else "EOA",
        "internal": True,
        "tokenTransfer": False,
        "blockNumber": tx['blockNumber']
    })

for tx in token_tx:
    combined.append({
        "txHash": tx['hash'],
        "from": tx['from'],
        "to": tx['to'],
        "value": tx['value'],
        "type_from": "Contract" if is_contract(tx['from']) else "EOA",
        "type_to": "Contract" if is_contract(tx['to']) else "EOA",
        "internal": False,
        "tokenTransfer": True,
        "blockNumber": tx['blockNumber']
    })

print(f"Total combined transactions: {len(combined)}")
